In [1]:
# dependencies
import pandas as pd
import numpy as np
import pymongo
from pymongo import MongoClient

KeyboardInterrupt: 

In [ ]:
# import csvs

# cleaned data

nuke_file = pd.read_csv("data/cleaned_nuke.csv")

countries_file = pd.read_csv("data/countries.csv")

happy_file = pd.read_csv("data/suicide.csv")

d3_file = pd.read_csv("data/nuked3.csv")

log_file = pd.read_csv("data/log_nukes.csv")

In [ ]:
# import to data bases
nuke_df = pd.DataFrame(nuke_file)

countries_df = pd.DataFrame(countries_file)

happy_df = pd.DataFrame(happy_file)

d3_df = pd.DataFrame(d3_file)

log_df = pd.DataFrame(log_file)

log_df.head()

In [ ]:
countries_df

In [ ]:
happy_df

In [ ]:
# drop country column in countries_df
countries_df = countries_df.drop(columns=["country"])

# countries_df.head()

In [ ]:
# renaming columns
countries_df = countries_df.rename(columns={"latitude":"Latitude", "longitude":"Longitude", "name":"Country"})
countries_df.head()

In [ ]:
merge_ltlng = pd.merge(nuke_df, countries_df, on='Country')

merge_ltlng.head()

In [ ]:
# drop unnamed index from merge_ltlng

lat_merged_df = merge_ltlng.drop(columns=["Unnamed: 0"])

lat_merged_df.head()

In [ ]:
# renaming columns for ease with database
lat_merged_df = lat_merged_df.rename(columns={"Quantity of Nuclear Weapons":"Quantity_of_Nuclear_Weapons",
                                            "GDP (Current LCU)":"GDP"})
lat_merged_df.head()


In [ ]:
happy_df = happy_df.rename(columns={"country-year":"Country_Year"})
happy_df["Country_Year"].unique() 



In [ ]:
# exporting a csv to combine with happiness indicators
# lat_merged_df.to_csv("data/nuke_coordinates_csv", index=False)

# Database Connection

In [ ]:
#Define connection link an instantiate client
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#Define the 'nukeDB' database in Mongo
db = client.nukeDB

#Declare the by_location collection
by_location = db.by_location
happyGeo = db.happyGeo
d3_collection = db.d3_collection
log_collection = db.log_collection

In [ ]:
#Instantiate for loop for populating business collection
for index, row in lat_merged_df.iterrows():
    #Populate post dictionary with information to be inserted into business collection
    post = {
        "Country_Year": str(row["Country_Year"]),
        "Country": str(row["Country"]),
        "Code": str(row["Code"]),
        "Year": int(row["Year"]),
        "Quantity_of_Nuclear_Weapons": int(row["Quantity_of_Nuclear_Weapons"]),
        "GDP": float(row["GDP"]),
        "Latitude":float(row["Latitude"]),
        "Longitude":float(row["Longitude"])
    }
    #Insert data into business collection, post{} by post{}
    by_location.insert_one(post)

In [ ]:
#Instantiate for loop for populating business collection
for index, row in d3_df.iterrows():
    #Populate post dictionary with information to be inserted into business collection
    post = {
        "Country_Year": str(row["Country_Year"]),
        "Country": str(row["Country"]),
        "Code": str(row["Code"]),
        "Year": int(row["Year"]),
        "Nukes": int(row["Nukes"]),
        "gdp_dollar": float(row["gdp_dollar"]),
        "color": str(row["color"])
    }
    #Insert data into business collection, post{} by post{}
    d3_collection.insert_one(post)

In [ ]:
#Instantiate for loop for populating business collection
for index, row in log_df.iterrows():
    #Populate post dictionary with information to be inserted into business collection
    post = {
        "Country_Year": str(row["Country_Year"]),
        "Country": str(row["Country"]),
        "Code": str(row["Code"]),
        "Year": int(row["Year"]),
        "Nukes": int(row["Nukes"]),
        "gdp_dollar": float(row["gdp_dollar"]),
        "color": str(row["color"]),
        "log_nukes": float(row["log_nukes"])
    }
    #Insert data into business collection, post{} by post{}
    log_collection.insert_one(post)

# Data to JSON For Visualizations

In [ ]:
lat_merged_df.to_json("data/coordinates_json.json")

# Bring in Geo and Happy Data

In [ ]:
lat_lon_file = pd.read_csv("data/world_lat_lon.csv")

geo_df = pd.DataFrame(lat_lon_file)
geo_df.head()

In [ ]:
geo_df = geo_df.rename(columns={"country":"Country", "latitude":"Latitude", "longitude":"Longitude"})
clean_geo_df = geo_df.drop(columns=["country_code","usa_state_code","usa_state_latitude","usa_state_longitude","usa_state"])
clean_geo_df = clean_geo_df[["Country","Latitude","Longitude"]]
clean_geo_df.head()

In [ ]:
happy_file = pd.read_csv("data/happy_2020.csv")
happyn_df = pd.DataFrame(happy_file)
happyn_df = happyn_df.rename(columns={"Country name":"Country","Social support":"Social_support","Healthy life expectancy":"Healthy_life_expectancy","Freedom to make life choices":"Freedom_to_make_life_choices","Perceptions of corruption":"Perceptions_of_corruption"})
happyn_df = happyn_df.drop(columns={"Regional indicator","Ladder score","Logged GDP per capita","Standard error of ladder score","upperwhisker","lowerwhisker","Ladder score in Dystopia","Explained by: Log GDP per capita","Explained by: Social support","Explained by: Healthy life expectancy","Explained by: Freedom to make life choices","Explained by: Generosity","Explained by: Perceptions of corruption","Dystopia + residual"})
happy_country_df = happyn_df.loc[happyn_df["Country"].isin(["China","France","India","Israel","Pakistan","Russia","United Kingdom","United States"])]
happy_country_df

In [ ]:
happy_geo_merge_df = pd.merge(clean_geo_df,happy_country_df, on="Country", suffixes=("_geo","_happy"))
happy_geo_merge_df["Healthy_life_expectancy"] = happy_geo_merge_df["Healthy_life_expectancy"].astype(float).map("{:,.2f}".format)
happy_geo_merge_df['Freedom_to_make_life_choices'] = pd.Series(["{0:.2f}%".format(val * 100) for val in happy_geo_merge_df['Freedom_to_make_life_choices']], index = happy_geo_merge_df.index)
happy_geo_merge_df['Perceptions_of_corruption'] = pd.Series(["{0:.2f}%".format(val * 100) for val in happy_geo_merge_df['Perceptions_of_corruption']], index = happy_geo_merge_df.index)
happy_geo_merge_df['Generosity'] = pd.Series(["{0:.2f}%".format(val * 100) for val in happy_geo_merge_df['Generosity']], index = happy_geo_merge_df.index)
happy_geo_merge_df['Social_support'] = pd.Series(["{0:.2f}%".format(val * 100) for val in happy_geo_merge_df['Social_support']], index = happy_geo_merge_df.index)
happy_geo_merge_df.head()

In [ ]:
happy_geo_merge_df.to_csv("data/happy_geo.csv", index=True)

In [ ]:
#Instantiate for loop for populating business collection
for index, row in happy_geo_merge_df.iterrows():
    #Populate post dictionary with information to be inserted into business collection
    post = {
        "Country": str(row["Country"]),
        "Latitude": float(row["Latitude"]),
        "Longitude": float(row["Longitude"]),
        "Healthy_life_expectancy": str(row["Healthy_life_expectancy"]),
        "Freedom_to_make_life_choices": str(row["Freedom_to_make_life_choices"]),
        "Generosity": str(row["Generosity"]),
        "Perceptions_of_corruption": str(row["Perceptions_of_corruption"])

    }
    #Insert data into business collection, post{} by post{}
    happyGeo.insert_one(post)

# HappyGeo data to JSON Visualization

In [ ]:
happy_geo_merge_df.to_json("data/happy_json.json")